In [1]:
import allel
import zarr
import numcodecs
import numpy as np
import sys
import matplotlib.pyplot as plt
from itertools import combinations
from collections import Counter

In [ ]:
# parse the command line arguements
# - validate the arguements 
# - throw errors if requiremabents are missing
# - validate the filter strings

pi_dxy --pi --dxy\ # must include at least 1 of pi and/dxy 
--vcf allsites.vcf.gz \ # required
--zarr path/to/zarr \  # default to the vcf folder
--populations popfile.txt \ # only required for dxy
--window_size 10000 \ # not required, defaults to whole genome
--snp_filter_expression "DP>=10, GQ>=20, FS>2" \ # required
--monomorphic_filter_expression "DP>=10, RGQ>=20" \ # required
--outfile pi_dxy_out.txt # default to vcf path + suffix

# https://stackoverflow.com/questions/40001892/reading-named-command-arguments
# example:
import argparse, sys

parser=argparse.ArgumentParser()

parser.add_argument('--bar', help='Do the bar option')
parser.add_argument('--foo', help='Foo the program')

args=parser.parse_args()

print args
print sys


In [ ]:
# validating inputs

# STEP 1 checking the vcf:
# - check if sci-kit allele can do this
# - check for contig info
# - alternatively use position vector
# - check for invariant sites and throw and error if they dont exist

# STEP 2 validate the population file
# - format is IND POP
# - throw an error if individuals are missing from VCF



In [ ]:
# filtering the data 

# - extracting the filtration values from the filter strings
# - applying the filter to the whole dataset
# - also filter out biallelic snps
# - check to see there is any data left
# - print warning for low data* think more about this


In [ ]:
# calculate pi

# for loop wrapper for pi calcuation
# output looks like:

# chromosome pos1 pos2 no_differences no_comparisons no_missing pi_1 pi_2 dxy
# chromosome pos1 pos2 no_differences no_comparisons no_missing pi
# chromosome pos1 pos2 no_differences no_comparisons no_missing dxy

# check if pi output makes sense
# - are any pis/dxys all zero?
# - check if # missing == (pos2 - pos1)
# - check if everything was either compared or missing
# - write out summary of program parameters file* think about how this should work
